In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [41]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [42]:
#  Loading data
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [43]:
df['price_value'].shape

(106494,)

In [44]:
#  Removal of EUR
df = df[ df['price_value'] != 'EUR' ]
df.shape

(106494, 155)

In [45]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [46]:
#  Removal of EUR
df = df[ df['price_currency'] != 'EUR' ]
df.shape

(106290, 155)

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue   #  If it's a list, continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values    #  If I have a feature that has been factorized before, it overwrites it
  else:
    df[feat + SUFFIX_CAT] = factorized_values   #  If it is not yet factorized, create it anew

In [0]:
#  Data cleaning, standardization
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ', '')))

In [0]:
def run_model(model, feats):
  X = df[feats].values            #  features
  y = df['price_value'].values    #  target variable

  #  Evaluate a score by cross-validation
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [50]:
#  Run XGBRegressor
feats = ['param_napęd__cat', 'param_rok-produkcji', 'param_stan__cat', 'param_skrzynia-biegów__cat', 'param_faktura-vat__cat', 'param_moc', 'param_marka-pojazdu__cat', 'feature_kamera-cofania__cat', 'param_typ__cat', 'seller_name__cat', 'param_pojemność-skokowa', 'feature_wspomaganie-kierownicy__cat', 'param_model-pojazdu__cat', 'param_wersja__cat', 'param_kod-silnika__cat', 'feature_system-start-stop__cat', 'feature_asystent-pasa-ruchu__cat', 'feature_czujniki-parkowania-przednie__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_regulowane-zawieszenie__cat']

xgb_params = { 
    'max_depth': 5, 
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 2020
}

run_model(xgb.XGBRegressor(**xgb_params, objective ='reg:squarederror'), feats)

(-9442.087634351257, 85.72564546598069)

## Hyperopt

In [51]:
#  Target function
def obj_func(params):
  print("Training with params: ")
  print(params)

  # Model training using cross-validation. The result is mean mean absolute error (mean mae)
  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  #  Returning a dictionary consisting of status information whether it was possible to calculate the iteration (ok) 
  #  and the value that the optimizer expects
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

#  Space: Specifying searchable range for hyperparameters
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',         np.arange(5, 16, 1, dtype=int)),
    'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':        'reg:squarederror',
    'n_estimators':     100,
    'seed':             2020,
}

#  Run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.15000000000000002, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 2020, 'subsample': 0.6000000000000001}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.15000000000000002, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 2020, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 2020, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.25, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 2020, 'subsample': 0.5}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.3, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 2020, 'subsample': 1.0}
Training with param

{'colsample_bytree': 0.7000000000000001,
 'learning_rate': 5,
 'max_depth': 6,
 'subsample': 1.0}